In [8]:
import numpy as np
from scipy.optimize import root
from scipy.stats import norm

def unsupervised_learning_threshold(V):
    # Normalize inlier count list
    V = V / np.max(V)

    # Parameters initialization
    µ = np.array([-2.0, 1.0])
    Σ = np.array([1.0, 1.0])
    π = np.array([0.5, 0.5])
    φ = np.array([µ, Σ, π])
    φ_prime = 0
    δ = 0.001  # Convergence threshold
    N = len(V)

    # EM algorithm
    while np.abs(φ_prime - φ).any() > δ:
        φ_prime = φ
        T = np.zeros((N, 2))

        # Expectation step
        for n in range(N):
            Zn = 0
            for j in range(2):
                Zn += π[j] * norm.pdf(V[n], µ[j], Σ[j])
            for k in range(2):
                T[n, k] = π[k] * norm.pdf(V[n], µ[k], Σ[k]) / Zn

        # Maximization step
        for k in range(2):
            µ[k] = np.sum(T[:, k] * np.log(V)) / np.sum(T[:, k])
            Σ[k] = np.sqrt(np.sum(T[:, k] * (np.log(V) - µ[k])**2) / np.sum(T[:, k]))
            π[k] = np.sum(T[:, k]) / N

        φ = np.array([µ, Σ, π])

    # Solve for the root vT in Equation 7
    def equation(vT):
        # Equation 7
        return π[0] * norm.pdf(vT, µ[0], Σ[0]) - π[1] * norm.pdf(vT, µ[1], Σ[1])

    vT_solution = root(equation, 0)
    vT = vT_solution.x[0]

    # Denormalization
    TGV = np.max(V) * vT

    return TGV




In [12]:
import numpy as np

with open('/mnt/sda3/Projects/Deep-VLCD-Benchmark/output/superglue_a/robotcar_qAutumn_dbNight_diff_final.txt', 'r') as f:
    lines = f.readlines()

inlier_counts = np.array([int(line.split(',')[-1]) for line in lines])

TGV = unsupervised_learning_threshold(inlier_counts)

print(TGV)

KeyboardInterrupt: 